##Descrição geral

Neste trabalho, implementamos, em Python, o algoritmo branch-and-bound para problemas de programação linear inteira binária.
Todas as funções objetivos dos testes são de maximização e todas as restrições são de "menor ou igual", com exceção daquelas que definem o domínio das variáveis.

##Regra de ramificação

Nosso código ramifica em torno da variável $x_{j}$ cujo valor é fracionário e o mais próximo possível de 0,5. Em um filho, adiciona-se a restrição $x_{j}$ = 1; no outro, a restrição $x_{j}$ = 0.

##Estratégia de ramificação

A estratégia de ramificação do nosso código segue a abordagem padrão do branch and bound.

##Dicionário de abreviações:

mlr_sol_atual = Melhor Solução Atual
objt = Objetivo


##Passo a passo

1 - Inicializa um nó raiz, representando o problema original

    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs)

2 - O modelo do problema inteiro é relaxado e resolvido.

    #passando a permitir valores fracionários e obtendo tanto uma solução inicial quanto um limite superior.

    def branch_and_bound(self, node)

    variables = [model.add_var(...]
    
    model.objective = maximize(....)

    for constraint_coeffs, rhs in zip(self.constraints_coeffs, self.constraints_rhs)

    model.optimize()

3 - Verifica se a solução é inteira

    #Se todas as variaveis tem valor 0 ou 1, o nó é considerado viável e para de ser ramificado

    if model.num_solutions

3.1 - Identificação de Fracionario

    Se alguma das variáveis contiver valores entre 0 e 1, ela é identificada e ramificada (fixada no nó filho 0 e no nó filho 1)

    frac_var = self.identify_branching_variable(model)
    
4 - Ramificação e Recursividade
    Seguindo de 3.1, os nós filhos são resolvidos pela mesma estrategia até que todas as variáveis tenham valores inteiros em pelo menos uma solução viável (ou até encontrar um limite inferior maior que o limite superior)

    if frac_var: (ramifica)

5 - Atualização de limites e da melhor solução

    if model.num_solutions:
      if model.objective_value > self.mlr_sol_atual:
        (atualiza)

In [ ]:
#@title instaladores
!pip install mip

In [34]:
#@title bibliotecas

from mip import Model, maximize,CONTINUOUS, BINARY, xsum

In [49]:
class Node:
    def __init__(self, problema, no_pai=None, branching_var=None, branch_value=None):
        #inicializando o nó
        self.problema = problema
        self.no_pai = no_pai
        self.branching_var = branching_var
        self.branch_value = branch_value

In [61]:
class BranchAndBound:
    def __init__(self, objective_coeffs, constraints_coeffs, constraints_rhs):
      #inicializando o problema
      self.objective_coeffs = objective_coeffs #coef das funcoes objetivo
      self.constraints_coeffs = constraints_coeffs #coefs das restricçoes do problema
      self.constraints_rhs = constraints_rhs #lados direitos das restricoes
      self.solucao_otima = None
      self.mlr_sol_atual = float('-inf')

    def solve(self):
        self.branch_and_bound(Node(problema=self))

    def branch_and_bound(self, node):
        model = Model()

        #variáveis de decisão
        variables = [model.add_var(name=f"x_{i}", var_type=BINARY) for i in range(len(self.objective_coeffs))]

        #função objt
        model.objective = maximize(xsum(self.objective_coeffs[i] * variables[i] for i in range(len(variables))))

        #organziando as restrições
        for constraint_coeffs, rhs in zip(self.constraints_coeffs, self.constraints_rhs):
            model += xsum(constraint_coeffs[i] * variables[i] for i in range(len(variables))) <= rhs

        model.optimize()

        if model.num_solutions:
            if model.objective_value > self.mlr_sol_atual:
                self.mlr_sol_atual = model.objective_value
                self.solucao_otima = {var.name: (var.x, self.objective_coeffs[int(var.name.split('_')[1])]) for var in variables}

            frac_var = self.identify_branching_variable(model)
            if frac_var:
                left_child = Node(problema=self, no_pai=node, branching_var=frac_var, branch_value=0)
                right_child = Node(problema=self, no_pai=node, branching_var=frac_var, branch_value=1)
                self.branch_and_bound(left_child)
                self.branch_and_bound(right_child)

    def identify_branching_variable(self, model):
        #para todas as variaveis no modelo atual
        for var in model.vars:
            #verifica se é inteiro ou fracionario
            if var.x != 0 and var.x != 1:
                return var
        return None


In [51]:
def read_data(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        #verifica quantas variáveis e restrições (linha 1)
        num_vars, num_constraints = map(int, lines[0].split())

        #lendo os coeficientes da função objt
        objective_coeffs = list(map(int, lines[1].split()))

        #lista para armazenar as restrições
        constraints_coeffs = []
        constraints_rhs = []

        #lendo e separando as restrições
        for line in lines[2:]:
            coeffs_and_rhs = list(map(int, line.split()))
            constraints_coeffs.append(coeffs_and_rhs[:-1])
            constraints_rhs.append(coeffs_and_rhs[-1])

    return num_vars, num_constraints, objective_coeffs, constraints_coeffs, constraints_rhs

In [64]:
for i in range(1, 5):
    file_name = f"teste{i}.txt"
    num_vars, num_constraints, objective_coeffs, constraints_coeffs, constraints_rhs = read_data(file_name)

    resolvedor = BranchAndBound(objective_coeffs, constraints_coeffs, constraints_rhs)
    resolvedor.solve()

    print(f"Teste {i}:")
    print("Melhor solução encontrada: ")
    print("x_{i} = (valor designado, coeficiente) ")
    for var_name, var_value in resolvedor.solucao_otima.items():
      print(f"{var_name} = {var_value}")
    print("Valor da função objetivo:", resolvedor.mlr_sol_atual)
    print()


Teste 1:
Melhor solução encontrada: 
x_{i} = (valor designado, coeficiente) 
x_0 = (0.0, 2)
x_1 = (0.0, 10)
x_2 = (0.0, 8)
x_3 = (0.0, 7)
x_4 = (1.0, 10)
x_5 = (1.0, 10)
x_6 = (0.0, 6)
Valor da função objetivo: 20.0

Teste 2:
Melhor solução encontrada: 
x_{i} = (valor designado, coeficiente) 
x_0 = (0.0, 7)
x_1 = (1.0, 7)
x_2 = (0.0, 7)
x_3 = (0.0, 5)
x_4 = (0.0, 8)
x_5 = (1.0, 8)
x_6 = (1.0, 9)
x_7 = (0.0, 10)
x_8 = (0.0, 7)
Valor da função objetivo: 24.0

Teste 3:
Melhor solução encontrada: 
x_{i} = (valor designado, coeficiente) 
x_0 = (0.0, 7)
x_1 = (0.0, 9)
x_2 = (1.0, 10)
x_3 = (0.0, 3)
x_4 = (0.0, 6)
x_5 = (0.0, 1)
x_6 = (1.0, 9)
x_7 = (0.0, 8)
x_8 = (0.0, 8)
Valor da função objetivo: 19.0

Teste 4:
Melhor solução encontrada: 
x_{i} = (valor designado, coeficiente) 
x_0 = (0.0, 9)
x_1 = (0.0, 7)
x_2 = (1.0, 10)
x_3 = (0.0, 7)
x_4 = (0.0, 9)
x_5 = (0.0, 6)
x_6 = (0.0, 8)
x_7 = (0.0, 4)
x_8 = (0.0, 9)
Valor da função objetivo: 10.0

